In this analysis, the gene expression data for some of the subjects in ADNI1/GO and ADNI2 phase is used to identify the most significantly different (in terms of their expression) between the research groups, example AD and CN, MCI and AD,etc.

For this analysis, ADNI_Gene_Expression_Profile is downloaded from the ADNI-Genetic_Data section. It has gene expression over 49386 probes for a total of 744 subjects.

The goal is to identify the genes with maximum discriminative value for Alzheimer's disease phase classification. In the end these gene expression values can be used as additional features to train the classifier which so far have been trained with neuroimaging data (eg. MRI) only.

Because the end goal is to use these gene expression features (if considered important), it makes sense to determine the presence of imaging data for which the gene expression is also available. For this a .csv file is downloaded using the Advanced Search (Beta) option where information of subjects from all ADNI phase, all image modalities are conslidated into a single file called ADNI_subjects.csv

Apart from genetic data, the demographic information, neuropsychological assessment results, etc available in ADNIMERGE.csv for all the subjects in ADNI are also used for analysis.

This notebook will combine following information into one csv file:
1. Subject's demographic information and assessment results from ADNIMERGE.csv
2. A uniform VisitID consistent across different modalities and csv files
3. Gene expression results extracted from Gene_Expression_Profile.csv
4. Availability of imaging data for given subject

As it should be obvious the first step towards this should be mapping the visit code for the files (VISCODE) which apparently has been labeled differently during different phases of the ADNI project. For this task ADNI2_VISITID.csv and VISITS.csv files downloaded from ADNI- Study_data section were used to remove the inconsistencies in VISIT code across different csv files, then the overall csv file  named as ('My_csv.csv') can be built accurately. 



One good thing is the file ADNIMERGE.csv has the visit code already resolved. All we need to do is the resolve the inconsistency in the visit code for subjects in ADNI2 for ADNI_Gene_Expression_Profile.csv file. Once VISCODE is resolved We can also extract the EXAMDATE information from ADNIMERGE.csv for the subjects with Gene expression profile. This would allow us to check the presence of right imaging file corresponding to the subjects and visit with gene expression data

First let us peek into our current working directory

In [9]:
import os
data_prep_path = '/Users/subashkhanal/Desktop/MLforAlzheimers/data/ADNI/genetics/data_prep'

In [10]:
os.listdir(data_prep_path)

['My_visCodes1.csv',
 'My_csv.csv',
 'ADNIMERGE.csv',
 '.DS_Store',
 'VISITS.xlsx',
 'ADNI_Gene_Expression_Profile.csv',
 'ADNI2_VISITID.csv',
 'ADNI_subjects_withVISCODE.csv',
 'ADNI_subjects.csv']

Now let's load all these files using pandas

In [11]:
import pandas as pd
gene_expr_data_path ='/Users/subashkhanal/Desktop/MLforAlzheimers/data/ADNI/genetics/ADNI_Gene_Expression_Profile/'

In [12]:
gene_exp = pd.read_csv(os.path.join(gene_expr_data_path,'ADNI_Gene_Expression_Profile.csv'),low_memory=False)


In [28]:
merge_df = pd.read_csv(os.path.join(data_prep_path,'ADNIMERGE.csv'),low_memory=False)
subjects_df = pd.read_csv(os.path.join(data_prep_path,'ADNI_subjects.csv'),low_memory=False)
ADNI2_VISITID = pd.read_csv(os.path.join(data_prep_path,'ADNI2_VISITID.csv'),low_memory=False)
my_pd = pd.read_csv(os.path.join(data_prep_path,'My_csv.csv'),low_memory=False)

As seen I already have created a template for My_csv.csv file. It contains subjects from gene expression file and selected columns from ADNIMERGE.csv file

In [29]:
my_pd.head()

,COLPROT,VISCODE,PTID,EXAMDATE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,...,ADAS13_bl,ADAS13,MMSE_bl,MMSE,DX_bl,DX,MRI,FDG_PET,AV45_PET,TAU_PET
0,ADNIGO,m48,116_S_1249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI2,v03,037_S_4410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADNI2,v03,006_S_4153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADNIGO,m48,116_S_1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADNI2,v03,099_S_4205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
set(my_pd[my_pd['COLPROT'] == 'ADNI2']['VISCODE'])

{'v02', 'v03', 'v04', 'v05', 'v06', 'v11'}

Now let's map the VISCODE for ADNI2 from vxx to mxx using RID and current VISCODE and looking up VISCODE2 in 'ADNI2_VISITID.csv'. 
Note PTID or subject ID is the form of: SiteID_S_RID

In [31]:
for i in range(len(my_pd)):
    phase = my_pd.iloc[i]['COLPROT']
    if phase == 'ADNI2':
        
        vid = my_pd.iloc[i]['VISCODE']
        RID = int(my_pd.iloc[i]['PTID'].split('_')[-1])
        #print (vid, RID)

        #look up VISCODE2 in mapping csv
        n_vid = ADNI2_VISITID[(ADNI2_VISITID['RID'] == RID) & (ADNI2_VISITID['VISCODE']==vid)]['VISCODE2'].iloc[0]
        
        my_pd.at[i,'VISCODE']=n_vid

In [32]:
set(my_pd[my_pd['COLPROT'] == 'ADNI2']['VISCODE'])

{'bl', 'm03', 'm06', 'm12', 'm48', 'm60', 'm72', 'm84', 'scmri'}

In [33]:
set(my_pd['COLPROT'])

{'ADNI2', 'ADNIGO'}

In [34]:
my_pd.head()

,COLPROT,VISCODE,PTID,EXAMDATE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,...,ADAS13_bl,ADAS13,MMSE_bl,MMSE,DX_bl,DX,MRI,FDG_PET,AV45_PET,TAU_PET
0,ADNIGO,m48,116_S_1249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI2,bl,037_S_4410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADNI2,bl,006_S_4153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADNIGO,m48,116_S_1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADNI2,bl,099_S_4205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now let's extract information from ADNIMERGE.csv for our subjects and visits

In [35]:
#columns in my csv for which we want information to be extracted from ADNIMERGE, last four columns are related to presence of imaging file
#those four would be filled using subjects_df downloaded from image search option 
merge_cols = my_pd.columns[:-4]

In [36]:
#just to see if all vist codes are also present in ADNIMERGE
set(my_pd['VISCODE']) <= set(merge_df['VISCODE']) #to check column name match or not

False

In [37]:
set(merge_df['VISCODE']),set(my_pd['VISCODE'])

({'bl',
  'm0',
  'm03',
  'm06',
  'm102',
  'm108',
  'm114',
  'm12',
  'm120',
  'm126',
  'm132',
  'm144',
  'm156',
  'm168',
  'm18',
  'm24',
  'm30',
  'm36',
  'm42',
  'm48',
  'm54',
  'm60',
  'm66',
  'm72',
  'm78',
  'm84',
  'm90',
  'm96',
  'y1'},
 {'bl', 'm03', 'm06', 'm12', 'm36', 'm48', 'm60', 'm72', 'm84', 'scmri'})

let's see what are the instances with visit code 'scrmi' as that one is not present in ADNIMERGE.csv

In [38]:
my_pd[my_pd['VISCODE']=='scmri']

,COLPROT,VISCODE,PTID,EXAMDATE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,...,ADAS13_bl,ADAS13,MMSE_bl,MMSE,DX_bl,DX,MRI,FDG_PET,AV45_PET,TAU_PET
360,ADNI2,scmri,009_S_4564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There is one instance with PTID '009_S_4564' with visit code 'scmri' that is not present in ADNIMERGE. It is seen from imaging record that the subject was screened for Imaging on date 3/9/2012 and labeled as "ADNI2 Screening MRI-New Pt". There is availability of gene expression data for this PTID taken on year 2012. and ADNI merge shows a baseline study 'bl' on date 3/20/2012. What does this say? Well, the imaging data and other forms of data may not have been collected on the same date!!

In [39]:
my_pd[my_pd['PTID']=='009_S_4564']

,COLPROT,VISCODE,PTID,EXAMDATE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,...,ADAS13_bl,ADAS13,MMSE_bl,MMSE,DX_bl,DX,MRI,FDG_PET,AV45_PET,TAU_PET
360,ADNI2,scmri,009_S_4564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
my_pd = my_pd.T


Index(['COLPROT', 'VISCODE', 'PTID', 'EXAMDATE', 'AGE', 'PTGENDER', 'PTEDUCAT',
       'PTETHCAT', 'PTRACCAT', 'APOE4', 'FDG_bl', 'FDG', 'ABETA_bl', 'ABETA',
       'TAU_bl', 'TAU', 'PTAU_bl', 'PTAU', 'Hippocampus_bl', 'Hippocampus',
       'WholeBrain_bl', 'WholeBrain', 'ADAS13_bl', 'ADAS13', 'MMSE_bl', 'MMSE',
       'DX_bl', 'DX', 'MRI', 'FDG_PET', 'AV45_PET', 'TAU_PET'],
      dtype='object')

In [40]:
my_pd = my_pd.astype({'EXAMDATE': 'str', 'PTGENDER': 'str','PTGENDER':'str', 'PTEDUCAT':'str',
       'PTETHCAT':'str', 'PTRACCAT':'str','DX_bl':'str','DX':'str','ABETA_bl':'str','ABETA':'str'})

In [50]:
#Insert entries from MERGE dataframe
for i in range(len(my_pd)):
    
    phase = my_pd['COLPROT'].iloc[i]
    visit = my_pd['VISCODE'].iloc[i]
    subject = my_pd['PTID'].iloc[i]
    extracted_row = merge_df[(merge_df['COLPROT']==phase)& (merge_df['VISCODE']==visit)& (merge_df['PTID']==subject)]
   
    if len(extracted_row) !=0 :
        
        for col in merge_cols:
            value = extracted_row[col].iloc[0]
            if (i==366) & (value == '<8'): #one bug case
                my_pd.at[i,col] = 7.99
            else:
                 my_pd.at[i,col] = value
        
    
    

In [63]:
my_pd.to_csv(os.path.join(data_prep_path,'my_csv_final1.csv'))

In [54]:
set(subjects_df['Visit'])

{'ADNI Baseline',
 'ADNI Screening',
 'ADNI1/GO Month 12',
 'ADNI1/GO Month 18',
 'ADNI1/GO Month 24',
 'ADNI1/GO Month 36',
 'ADNI1/GO Month 48',
 'ADNI1/GO Month 6',
 'ADNI2 Baseline-New Pt',
 'ADNI2 Initial Visit-Cont Pt',
 'ADNI2 Month 3 MRI-New Pt',
 'ADNI2 Month 6-New Pt',
 'ADNI2 Screening MRI-New Pt',
 'ADNI2 Tau-only visit',
 'ADNI2 Year 1 Visit',
 'ADNI2 Year 2 Visit',
 'ADNI2 Year 3 Visit',
 'ADNI2 Year 4 Visit',
 'ADNI2 Year 5 Visit',
 'ADNI3 Initial Visit-Cont Pt',
 'ADNI3 Year 1 Visit',
 'ADNI3 Year 2 Visit',
 'ADNI3 Year 3 Visit',
 'ADNIGO Month 3 MRI',
 'ADNIGO Month 60',
 'ADNIGO Month 72',
 'ADNIGO Screening MRI',
 'No Visit Defined',
 'Unscheduled'}

Now let's copy the information about presence of imaging file. Using the subject_df. Now it would have been nicer if the imaging files were also tagged with a VISCODE but instead they have description. So we need to get a VISCODE using the Visit description 
in the subject_df. For this a mapping file called 'My_visCodes1.csv was created using the visit code related files available in ADNI and used here to create a visit code for each imaging file.

In [55]:
# my_vis = pd.read_excel('My_visCodes.xlsx')
# my_vis['VISNAME'] = list(set(subjects_df['Visit']))


In [56]:
#my_vis.to_excel('My_visCodes1.xlsx')

In [57]:
my_vis = pd.read_csv(os.path.join(data_prep_path,'My_visCodes1.csv'))
subjects_df = pd.read_csv(os.path.join(data_prep_path,'ADNI_subjects.csv'))

Now using the map "My_visCodes1.csv' Let's fill the column VISCODE in the subjects_df

In [58]:
subjects_df = subjects_df.astype({'VISCODE': 'str'}) 

In [59]:
for i in range(len(subjects_df)):
    visname = subjects_df['Visit'].iloc[i]
    viscode = my_vis[my_vis['VISNAME']==visname]['VISCODE'].iloc[0]
    subjects_df.at[i,'VISCODE'] = viscode
    

In [60]:
set(subjects_df['VISCODE'])

{'bl',
 'init',
 'm03',
 'm06',
 'm12',
 'm18',
 'm24',
 'm36',
 'm48',
 'm60',
 'm72',
 'nv',
 'sc',
 'scmri',
 'tau',
 'uns',
 'v02',
 'v03',
 'v04',
 'v05',
 'v06',
 'v11',
 'v21',
 'v31',
 'v41',
 'v51',
 'y1',
 'y2',
 'y3'}

However, this still has visit codes like vxx for ADNI2. So let's change those to mxx form as we did earlier

In [61]:
for i in range(len(subjects_df)):
    phase = subjects_df.iloc[i]['Phase']
    if phase == 'ADNI 2':
        
        vid = subjects_df.iloc[i]['VISCODE']
        RID = int(subjects_df.iloc[i]['Subject ID'].split('_')[-1])
        #print (vid, RID)
        if vid in set(ADNI2_VISITID['VISCODE']): #to avoid cases like vid= 'tau'
            
            #look up VISCODE2 in mapping csv
            n_vid = ADNI2_VISITID[(ADNI2_VISITID['RID'] == RID) & (ADNI2_VISITID['VISCODE']==vid)]['VISCODE2']
            n_vid = list(n_vid)[0]



            subjects_df.at[i,'VISCODE']=n_vid

In [0]:
subjects_df.to_csv(os.path.join(data_prep_path,'ADNI_subjects_withVISCODE.csv')

In [64]:
my_pd = pd.read_csv(os.path.join(data_prep_path,'my_csv_final1.csv'))
subjects_df= pd.read_csv(os.path.join(data_prep_path,'ADNI_subjects_withVISCODE.csv'))

At this point we have all the files with consistent visit code. Now let's fill my_csv with presence of imaging data information

In [68]:
my_pd.columns

Index(['Unnamed: 0', 'COLPROT', 'VISCODE', 'PTID', 'EXAMDATE', 'AGE',
       'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'APOE4', 'FDG_bl',
       'FDG', 'ABETA_bl', 'ABETA', 'TAU_bl', 'TAU', 'PTAU_bl', 'PTAU',
       'Hippocampus_bl', 'Hippocampus', 'WholeBrain_bl', 'WholeBrain',
       'ADAS13_bl', 'ADAS13', 'MMSE_bl', 'MMSE', 'DX_bl', 'DX', 'MRI',
       'FDG_PET', 'AV45_PET', 'TAU_PET'],
      dtype='object')

In [69]:
my_pd = my_pd.assign(MRI=0)
my_pd = my_pd.assign(FDG_PET=0)
my_pd = my_pd.assign(AV45_PET=0)
my_pd = my_pd.assign(TAU_PET=0)

In [70]:
for i in range(len(my_pd)):
    
    ptid = my_pd.iloc[i]['PTID']
    phase = 'ADNI ' + my_pd.iloc[i]['COLPROT'][4:] #to accomodate for space in subject_df
    group = my_pd.iloc[i]['DX'] #or DX_bl
    matching_images = subjects_df[(subjects_df['Subject ID'] == ptid) & (subjects_df['Phase'] == phase) & (subjects_df['Research Group'] == group)]
    if 'MRI' in set(matching_images['Modality']):
        my_pd.at[i,'MRI']= 1
    
    if 'PET' in set(matching_images['Modality']):
        images_descr = list(set(matching_images['Description']))
        for pet in images_descr:
            if 'FDG' in pet:
                my_pd.at[i,'FDG_PET']= 1
            
            if 'AV45' in pet:
                my_pd.at[i,'AV45_PET']= 1
                
            if 'Tau' in pet:
                my_pd.at[i,'TAU_PET']= 1
        
    

For how many subjects with genetic expression data corresponding modalities of imaging data is available (For either baseline diagnosis or current diagnosis

Current Diagnosis(DX) OR Baseline Diagnosis(DX_bl)

MRI = 649:
        DX_bl = Counter({'CN': 229, 'AD': 40, 'EMCI': 179, 'LMCI': 201})
        DX= Counter({'CN': 216, 'Dementia': 102, 'MCI': 323, nan: 8})
        
AV45_PET = 599:
        DX_bl = Counter({'AD': 37, 'CN': 208, 'EMCI': 163, 'LMCI': 191})
        DX= Counter({'Dementia': 97, 'CN': 195, 'MCI': 300, nan: 7})


FDG_PET = 610:
        DX_bl = Counter({'AD': 38, 'CN': 210, 'EMCI': 168, 'LMCI': 194})
        DX= Counter({'Dementia': 98, 'CN': 197, 'MCI': 307, nan: 8})


TAU_PET = 39:
        DX_bl = Counter({'AD': 5, 'EMCI': 13, 'CN': 16, 'LMCI': 5})
        DX= Counter({'Dementia': 5, 'MCI': 20, 'CN': 12, nan: 2})


But genetic expression changes over time so to be more accurate the count of imaging corresponding to only the current diagnosis is:

for MRI: Counter({'MCI': 148, 'CN': 94, 'Dementia': 40, nan: 3}) Total:  285

for AV45_PET: Counter({'MCI': 131, 'CN': 79, 'Dementia': 36, nan: 3}) Total:  249

for FDG_PET: Counter({'MCI': 137, 'CN': 79, 'Dementia': 37, nan: 3}) Total:  256

for TAU_PET: Counter({'MCI': 10, 'CN': 8, 'Dementia': 3, nan: 1}) Total:  22


In [71]:
print('for MRI:',Counter(my_pd[my_pd['MRI']==1]['DX']),'Total: ',my_pd['MRI'].sum())
print('for AV45_PET:',Counter(my_pd[my_pd['AV45_PET']==1]['DX']),'Total: ',my_pd['AV45_PET'].sum())
print('for FDG_PET:',Counter(my_pd[my_pd['FDG_PET']==1]['DX']),'Total: ',my_pd['FDG_PET'].sum())  
print('for TAU_PET:',Counter(my_pd[my_pd['TAU_PET']==1]['DX']),'Total: ',my_pd['TAU_PET'].sum())

for MRI: Counter({'CN': 228, 'MCI': 57}) Total:  285
for AV45_PET: Counter({'CN': 205, 'MCI': 44}) Total:  249
for FDG_PET: Counter({'CN': 210, 'MCI': 46}) Total:  256
for TAU_PET: Counter({'CN': 22}) Total:  22


At this point my_pd has data from ADNIMERGE and information about  presence of imaging data

Now let's add the gene expression data on it!!

In [72]:
probes = gene_exp['Phase'][8:]
len(probes)

49386

In [73]:
exps = gene_exp.iloc[8:,3:-1]

exps.shape

(49386, 744)

In [74]:
header_list = list(my_pd.columns) + list(probes)
my_pd = my_pd.reindex(columns = header_list) 
my_pd = my_pd.T
len(my_pd.columns),my_pd.shape,exps.shape

(744, (49419, 744), (49386, 744))

In [79]:
49419-49386

33

In [80]:
import numpy as np
my_pd.iloc[33:,:] = np.array(exps.iloc[:,:]) #copy the gene expression values for each subjects

In [82]:
my_pd.to_csv(os.path.join(data_prep_path,'ADNI_Imaging_Gene_Expression_MMSE_Combined.csv'))